In [ ]:
import re
import os
import random
import warnings
import chromadb
import gradio as gr
import pandas as pd
from datetime import datetime
from typing import List, Pattern
from langchain_chroma import Chroma
from langchain_huggingface.llms import HuggingFacePipeline
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate,
    SystemMessagePromptTemplate   
)
from langchain_huggingface import (
    ChatHuggingFace
    , HuggingFaceEmbeddings
    , HuggingFaceEndpoint
)
from transformers import logging as transformers_logging

warnings.simplefilter(action='ignore', category=FutureWarning)

pd.set_option('display.max_colwidth', None)

transformers_logging.set_verbosity_info()



**Definição dos locais onde se encontram a base de dados vetorial, modelos de embeddings e LLM e ficheiro com o resultado das avaçiações ao modelo**

In [ ]:
chroma_db_dir = "lexclaraDB/ChromaDB"
model_llm_name = "mistralai/Mistral-7B-Instruct-v0.3"
embeddings_name = "BAAI/bge-m3"

hf_token = os.getenv("HF_TOKEN")

**Inicialização do modelo de embeddings local**

In [ ]:

# documentar as definições do modelo de embeddings
model_kwargs = {'device': 'cuda'
                , 'trust_remote_code': True
                }

encode_kwargs = {'normalize_embeddings': True}

hf_embeddings = HuggingFaceEmbeddings(
    model_name=embeddings_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)


2025-10-29 19:54:43,949 - INFO - Load pretrained SentenceTransformer: /nelson/LexClara/models/huggingface/BAAI-bge-m3
loading configuration file /nelson/LexClara/models/huggingface/BAAI-bge-m3/config.json
Model config XLMRobertaConfig {
  "architectures": [
    "XLMRobertaModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 8194,
  "model_type": "xlm-roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.52.4",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

loading weights file /nelson/LexClara/models/huggingface/BAAI-bge-m3/pytorch_model.bin
All model che

**Inicialização da base de dados vetorial - ChromaDB**

In [ ]:
# Inicializar cliente persistente
client = chromadb.PersistentClient(path=chroma_db_dir)

# Nome da coleção
collection_name = "LexClara_bge_m3_1024" # documentos breves

colecao = client.get_collection(name=collection_name)

# Conectar à coleção com LangChain
vectordb = Chroma(
    client=client,
    collection_name=collection_name,
    embedding_function=hf_embeddings,
    persist_directory=chroma_db_dir
)

2025-10-29 19:55:45,126 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2025-10-29 19:55:45,139 - INFO - Coleção 'LexClara_bge_m3_1024' existente carregada.


**Início das funções auxiliares para o LExClaraBot**

**Função que permite reconhecer padrões no texto colocado na pergunta ao LLM**

Neste momento só permite o reconhecimento de dois tipos de diploma &mdash; Decreto-Lei e Decreto Regulamentar &mdash; correspondentes aos diplomas que possuem linguagem clara, extraídos do DRE-Tretas.

In [10]:

PADROES = [
    re.compile(r'(Decreto[-\s]?Lei)\s*(?:n\.º\s*)?(\d+(?:[A-Za-z-]*/\d{4}))', re.IGNORECASE),
    # re.compile(r'(Lei)\s*(?:n\.º\s*)?(\d+(?:[A-Za-z-]*/\d{4}))', re.IGNORECASE),
    re.compile(r'(Decreto\s+Regulamentar)\s*(?:n\.º\s*)?(\d+(?:[A-Za-z-]*/\d{4}))', re.IGNORECASE),
    # re.compile(r'(Portaria)\s*(?:n\.º\s*)?(\d+(?:[A-Za-z-]*/\d{4}))', re.IGNORECASE),
]

def extrair_por_regex(pergunta: str, patterns: List[Pattern]) -> List[str]:
    """
    Extrai do texto todos os identificadores completos (tipo + número),
    sem o 'n.º'. Exemplo de saída: ['Decreto-Lei 137/2023', 'Lei 12/2022'].
    """
    resultados = []
    for pat in patterns:
        for match in pat.finditer(pergunta):
            tipo = match.group(1).strip()
            numero = match.group(2).strip()
            resultados.append(f"{tipo} {numero}")
    return resultados

**Carregar o dataset de linguagem clara_2 para obter um diploma aleatório para testar as respostas do LLM**

In [ ]:
try:
    df_linguagem_clara_2 = pd.read_csv(filepath_or_buffer=os.path.join(os.getcwd(),'data','gold','linguagem-clara-2020-2024_2.csv'))
except Exception as e:
        print(f"Ocorreu o seguinte erro: {e}")

2025-10-29 19:56:01,877 - INFO - Dados do ficheiro /nelson/LexClara/data/gold/linguagem-clara-2020-2024_2.csv foram carregados com sucesso.


**Função para escolher um exemplo aleatório**

In [13]:

def gera_exemplo_aleatorio_do_df(df):

    tuplo_per_res=[]

    linha = df.sample(n=1).iloc[0]

    perguntas=[
            f"O que é o diploma {linha['identificacao_diploma']}?",
            f"O que vai mudar com o diploma {linha['identificacao_diploma']}?",
            f"Que vantagens traz o diploma {linha['identificacao_diploma']}?",
            f"Quando entra em vigor o diploma {linha['identificacao_diploma']}?"
        ]

    for i, perg in enumerate(perguntas):

        if i==0:
            res=linha['o_que_e']

        if i==1:
            res=linha['o_que_vai_mudar']

        if i==2:
            res=linha['que_vantagens_traz']
            
        if i==3:
            res=linha['quando_entra_em_vigor']

        tuplo_per_res.append((perg, res)) #type:ignore

    return tuplo_per_res


**Permite usar o exemplo aleatório no interface criado no Gradio**

In [14]:

def gera_exemplo_aleatorio_para_gradio():

    pergunta, resposta = random.choice(gera_exemplo_aleatorio_do_df(df=df_linguagem_clara_2))
    
    return pergunta, resposta

**Definição do _prompt_ de forma a ajudar o LLM a responder melhor às perguntas colocadas**

In [15]:
def construir_prompt_few_shot() -> ChatPromptTemplate:


    # 1. Definição das mensagens de sistema e de utilizador
    sys_message = SystemMessagePromptTemplate.from_template(
        "És um chatbot de assistência jurídica que ajuda o utilizador leigo a ter contacto com as leis portuguesas.\
        Responde à pergunta com base no contexto legislativo existente na base de dados. \
        Escreve frases completas com escrita e pontuação corretas. \
        Se o contexto não contiver informação suficiente, responde exatamente:  \
        Não há informação relevante nos diplomas selecionados. \
        Não inventes respostas que não estejam no contexto. \
        Os exemplos de diálogo apresentados servem apenas para demonstrar o formato e o estilo da resposta esperada, não devem ser repetidos. \
        ")


    exemplos = gera_exemplo_aleatorio_do_df(df=df_linguagem_clara_2)

    mensagens_exemplo = []

    for entrada, resposta in exemplos:
        mensagens_exemplo.append(HumanMessagePromptTemplate.from_template(entrada))
        mensagens_exemplo.append(AIMessagePromptTemplate.from_template(resposta))

    input_prompt = HumanMessagePromptTemplate.from_template(
        "Contexto legislativo relevante:\n{context}\n\n \
        Pergunta:\n{input}\n\n \
        Resposta")

    return ChatPromptTemplate.from_messages(
        [sys_message] + mensagens_exemplo + [input_prompt]
    )

**Criação das definições do LLM**

In [ ]:
def criar_llm(
    temperature: float,
    top_k: int,
    top_p: float,
    max_tokens: int,
    repetition_penalty: float
):
    """
    Cria e retorna um LLM da Mistral com os parâmetros ajustáveis.

    """

    repo_id = "mistralai/Mistral-7B-Instruct-v0.3"



    generator = HuggingFaceEndpoint(
        repo_id=repo_id
        ,task="text-generation"
        ,temperature=temperature
        ,top_k=top_k
        ,top_p=top_p
        ,max_new_tokens=max_tokens
        ,repetition_penalty=repetition_penalty
        ,huggingfacehub_api_token=hf_token
        ,do_sample=True
        ,streaming=True
        ,return_full_text=False
    )

    llm = ChatHuggingFace(llm=generator)
    
    return llm

**Função auxiliar que permite a recolha do contexto e lidar com casos sem documentos retornados**

In [ ]:
def responder_pelo_gradio_com_LLM(
    pergunta: str,
    temperature: float,
    top_p: float,
    top_k: int,
    max_tokens: int,
    repetition_penalty: float,
):

    # Extrai os identificadores de diploma da pergunta, para serem utilizado como critério de filtragem de documentos

    ids = extrair_por_regex(pergunta, PADROES)

    if not ids:
        return "Nenhum identificador de diploma encontrado na pergunta."
        exit

    retriever = vectordb.as_retriever(search_type="similarity"
                                    , search_kwargs={
                                        'filter': {
                                            'diploma': {'$in': ids}  # O mesmo filtro de metadados
                                        },
                                        'k': top_k  # O mesmo número de documentos a serem retornados
                                    }
                                )
    
    # Recupera documentos
    retrieved_docs = retriever.invoke(pergunta)
    if not retrieved_docs:
        contexto = "Nenhum contexto legislativo relevante foi encontrado para os diplomas mencionados."
        chunks = ""
    else:
        contexto = "\n\n".join([doc.page_content for doc in retrieved_docs])
        chunks = "\n\n---\n\n".join([doc.page_content for doc in retrieved_docs])

    # Prepara o LLM com os parâmetros recebidos, e presentes no interface
    llm = criar_llm(temperature
                    , top_k
                    , top_p
                    , max_tokens
                    , repetition_penalty)
    
    # construção do prompt de auxílio ao LLM para as respostas
    prompt = construir_prompt_few_shot()


    # Cria a chain de combinação de documentos (stuff)
    combine_docs_chain = create_stuff_documents_chain(
        llm=llm
        ,prompt=prompt
    )

    chain = create_retrieval_chain(
        retriever=retriever,
        combine_docs_chain=combine_docs_chain
    )

    result = chain.invoke({
        "context": contexto,
        "input": pergunta})


        
    # A query_dict virá da chain e conterá 'pergunta'
    retrieved_docs = retriever.invoke(pergunta)
    
    chunks = (
        "\n\n---\n\n".join([doc.page_content for doc in retrieved_docs])
        if retrieved_docs
        else "Não foram encontrados documentos que correspondem ao critério."
    )

    resposta = result.get("answer") or result.get("output_text") or str(result)

    return resposta, chunks

**Geração do interface com utilizado, via Gradio**

In [ ]:

with gr.Blocks(title="Pergunte à Legislação com Mistral", theme=gr.themes.Default(text_size="lg")) as chatbot_LexClara: # type: ignore


    gr.Markdown("##Chat Jurídico com Mistral (Few-shot + Parametrização)")
    
    with gr.Row():
        pergunta_input = gr.Textbox(label="Pergunta"
                                    , lines = 2
                                    , placeholder="Ex: O que é o Decreto-Lei n.º 137/2023?")
        
        usar_exemplo_btn = gr.Button("Usar Exemplo Aleatório")
    
    with gr.Accordion("Parâmetros Avançados", open=False):
        temperature = gr.Slider(minimum=0
                                , maximum=1
                                , value=0.7
                                , step=0.1
                                , label="Temperatura")
        
        top_p = gr.Slider(minimum=0
                          , maximum=1
                          , value=1.0
                          , step=0.05
                          , label="Top-p")
        
        top_k = gr.Slider(minimum=1
                          , maximum=50
                          , value=5
                          , step=1
                          , label="Top-k")
        
        max_tokens = gr.Slider(minimum=100
                               , maximum=2000
                               , value=512
                               , step=100
                               , label="Número máximo de tokens gerados")
        
        repetition_penalty = gr.Slider(minimum=1.0
                                       , maximum=2.0
                                       , value=1.2
                                       , step=0.1
                                       , label="Penalização por repetição")
    
    with gr.Row():
        resposta_output = gr.Textbox(label="Resposta do LLM", lines=4)
        chunks_output = gr.Textbox(label="Segmentos de Texto Recuperados", lines=8)

    resposta_esperada_output = gr.Textbox(label="Resposta Esperada (para avaliação)", lines=4)

    perguntar_btn = gr.Button("Obter Resposta")

    # Funções aplicadas aos botões
    usar_exemplo_btn.click(
        gera_exemplo_aleatorio_para_gradio,
        inputs=[],
        outputs=[pergunta_input
                , resposta_esperada_output]
    )

    perguntar_btn.click(
        responder_pelo_gradio_com_LLM,
        inputs=[pergunta_input
                , temperature
                , top_p
                , top_k
                , max_tokens
                , repetition_penalty],
        outputs=[resposta_output
                , chunks_output]
    )


2025-10-29 19:56:58,791 - INFO - HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"


In [21]:
chatbot_LexClara.launch(server_name="0.0.0.0"
                        , server_port=8085
                        , share=True
                        , debug=True)

2025-10-29 19:57:03,049 - INFO - HTTP Request: GET http://localhost:8085/gradio_api/startup-events "HTTP/1.1 200 OK"
2025-10-29 19:57:03,062 - INFO - HTTP Request: HEAD http://localhost:8085/ "HTTP/1.1 200 OK"


* Running on local URL:  http://0.0.0.0:8085


2025-10-29 19:57:03,796 - INFO - HTTP Request: GET https://api.gradio.app/v3/tunnel-request "HTTP/1.1 200 OK"


* Running on public URL: https://0c1863bf4a5f81333e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


2025-10-29 19:57:05,867 - INFO - HTTP Request: HEAD https://0c1863bf4a5f81333e.gradio.live "HTTP/1.1 200 OK"


Keyboard interruption in main thread... closing server.
Killing tunnel 0.0.0.0:8085 <> https://0c1863bf4a5f81333e.gradio.live
